In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import re
from pathlib import Path
import numpy as np
import pandas as pd

In [3]:
PROJ = Path(os.path.realpath("."))
ROOT = PROJ.parent
DATA = ROOT / "data"

In [4]:
sys.path.append(str(ROOT / "src"))

In [5]:
from get_emeddings import prepare_data_and_embeddings

# Read data

In [6]:
# Classification A
clas_a_df = pd.read_excel(DATA / "raw/Categories 2019_2018.xlsx")
clas_a_df = clas_a_df[["MAIN_TAXABLE_ACTIVITY 2018", "MAIN_TAXABLE_ACTIVITY 2019"]]
# Create one unique classification
clas_a_df = pd.DataFrame(
    clas_a_df["MAIN_TAXABLE_ACTIVITY 2018"]
    .append(clas_a_df["MAIN_TAXABLE_ACTIVITY 2018"])
    .unique(),
    columns=["clas_a_title"],
)
# Remove text in parantheses
clas_a_df["clas_a_title"] = clas_a_df["clas_a_title"].str.replace(r"\(.*\)", "").str.strip()
# Export
clas_a_df.head()

,clas_a_title
0,Business Support Services
1,Fishing
2,Agriculture Employment
3,Animal production
4,Support activities of Mining


In [7]:
# Classification B
sheets = ["HS2", "HS4", "NAICS2", "NAICS4"]

clas_b_df = pd.DataFrame()
for sheet in sheets:
    clas_b_df_sheet = pd.read_excel(
        DATA / "raw/codes.xlsx", sheet_name=sheet, dtype=str
    )
    clas_b_df_sheet.columns = ["clas_b_code", "clas_b_title"]
    clas_b_df_sheet["clas_b_type"] = sheet
    clas_b_df = clas_b_df.append(clas_b_df_sheet)
clas_b_df["clas_b_code"] = clas_b_df["clas_b_type"] + "_" + clas_b_df["clas_b_code"]
clas_b_df.head()

,clas_b_code,clas_b_title,clas_b_type
0,HS2_1,Live animals,HS2
1,HS2_2,Meat,HS2
2,HS2_3,Fish,HS2
3,HS2_4,Diary products,HS2
4,HS2_5,Animal products,HS2


# Get embeddings

In [9]:
prepare_data_and_embeddings(
    clas_a_df,
    clas_b_df,
    titlecol_a="clas_a_title",
    titlecol_b="clas_b_title",
    codecol_b="clas_b_code",
)


Pre-processing text


[nltk_data] Downloading package punkt to /Users/shg309/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/shg309/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shg309/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
